In [1]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
import pandas as pd
from backtesting.test import SMA
import pandas as pd
from bokeh.io import export_png
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting import Position

/Users/fahad/anaconda3/lib/python3.7/site-packages/backtesting/_plotting.py:34: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `bokeh.io.reset_output()`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
MSFT_predictions = pd.read_csv("/Users/fahad/project_repo/models/final_models/Financial_Analysis/predictions/MSFT_predictions.csv", index_col=0)


In [3]:
MSFT_predictions.head()

,Date,Predictions
0,2019-06-28,0
1,2019-05-31,0
2,2019-04-30,1
3,2019-03-29,1
4,2019-02-28,1


In [4]:
MSFT_predictions.Date = pd.to_datetime(MSFT_predictions.Date, format="%Y-%m-%d")

In [5]:
MSFT_predictions['Predictions'] = MSFT_predictions['Predictions'].shift(-1)


In [6]:
MSFT_predictions = MSFT_predictions.dropna()


In [7]:
MSFT_predictions = MSFT_predictions[::-1]

In [8]:
MSFT_predictions.head(10)

,Date,Predictions
111,2010-03-31,1.0
110,2010-04-30,0.0
109,2010-05-28,0.0
108,2010-06-30,1.0
107,2010-07-30,0.0
106,2010-08-31,1.0
105,2010-09-30,1.0
104,2010-10-29,1.0
103,2010-11-30,1.0
102,2010-12-31,1.0


In [9]:
MSFT_monthly = pd.read_csv("/Users/fahad/project_repo/data/external/MSFT_monthly.csv", index_col =0)

In [10]:
MSFT_monthly.columns = ['Open','High','Low','Close','Volume']

In [11]:
MSFT_predictions = MSFT_predictions.set_index("Date")

In [12]:
MSFT_monthly.head()

,Open,High,Low,Close,Volume
2019-07-26,136.63,141.675,134.67,141.34,412494800
2019-06-28,123.85,138.400,119.01,133.96,508324017
2019-05-31,130.53,130.650,123.04,123.68,547217986
2019-04-30,118.95,131.370,118.10,130.60,433157427
2019-03-29,112.89,120.820,108.80,117.94,589044900


In [13]:
MSFT_final_df = pd.merge(MSFT_predictions, MSFT_monthly, right_index=True, left_index=True)

In [14]:
MSFT_final_df.index = pd.to_datetime(MSFT_final_df.index, format ="%Y-%m-%d")

In [15]:
equity_dict = dict()

In [16]:
class BH(TrailingStrategy):

    
    def init(self):
        
        super().init()
        self.set_trailing_sl(4)
        self.start = self.data.index[0]
        
    def next(self):
        super().init()
        equity_dict[self.data.index[-1]] = TrailingStrategy.equity.__get__(self)
        
        position = Strategy.position.__get__(self)
#         print (type(self.start))
        if self.data.index[-1] > self.start and (position.is_long!=True):
        
            self.buy()
        else:
            pass

In [17]:
bt = Backtest(MSFT_final_df, BH, cash=10000, commission=.01)
bt.run()



Start                     2010-03-31 00:00:00
End                       2019-06-28 00:00:00
Duration                   3376 days 00:00:00
Exposure [%]                                0
Equity Final [$]                      43245.4
Equity Peak [$]                       43245.4
Return [%]                            332.454
Buy & Hold Return [%]                 357.397
Max. Drawdown [%]                    -25.7184
Avg. Drawdown [%]                    -7.65399
Max. Drawdown Duration      670 days 00:00:00
Avg. Drawdown Duration      171 days 00:00:00
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                            NaN
Max. Trade Duration                       NaT
Avg. Trade Duration                       NaT
Expectancy [%]                            NaN
SQN                                       NaN
Sharpe Ratio                      

In [18]:
bt.plot(superimpose=False, plot_volume=False)

In [19]:
df = pd.DataFrame.from_dict([equity_dict])

In [20]:
df = df.T

In [21]:
df.columns = ['Equity']

In [22]:
export = df.to_csv("/Users/fahad/project_repo/data/final/Equity/BH/MSFT_BH_Equity.csv")

In [23]:
df

,Equity
2010-04-30,10000.000000
2010-05-28,8328.840709
2010-06-30,7428.163749
2010-07-30,8332.068942
2010-08-31,7575.048343
2010-09-30,7905.942208
2010-10-29,8608.082849
2010-11-30,8153.709078
2010-12-31,9009.997837
2011-01-31,8950.275530
